Imports

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2 as pg
import warnings
from tkinter import *
from tkinter.filedialog import askopenfilename
import os
from dotenv import load_dotenv
warnings.filterwarnings("ignore")


Choose and open sheet

In [20]:
Tk().withdraw()
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(f"Nazwa pliku to: {filename}")
xls = pd.read_csv(filename)

Nazwa pliku to: C:/Users/Dawid Krysa/Downloads/lista_operacji_250101_250126_202501261646088931.csv


Connect to database

In [44]:
# Read data from .env file
load_dotenv()

# Connect to an existing database
conn = pg.connect(f"dbname={os.getenv('POSTGRES_DB')} user={os.getenv('POSTGRES_USER')} password={os.getenv('POSTGRES_PASSWORD')}")

# Open a cursor to perform database operations
cur = conn.cursor()


Execute a test query

In [45]:
cur.execute("SELECT * FROM categories WHERE main_category_id IS NOT NULL")
rows = cur.fetchall()
for row in rows:
    print(row)

(6, 'Poduszka finansowa', 1, False)
(7, 'Wkład własny', 1, False)
(8, 'Portfel długoterminowy', 1, False)
(9, 'Najem i media', 2, False)
(10, 'Zakupy spożywcze', 2, False)
(11, 'Internet', 2, False)
(12, 'Telefon', 2, False)
(13, 'Transport', 2, False)
(14, 'Serwisy streamingowe', 2, False)
(15, 'Kosmetyki, drogera i środki czystości', 2, False)
(16, 'Nieprzewidziane wydatki', 2, False)
(17, 'Leasing', 3, False)
(18, 'Księgowy', 3, False)
(19, 'Kredyt gotówkowy', 3, False)
(20, 'Fryzjer', 4, False)
(21, 'Ubrania', 4, False)
(22, 'Jedzenie na mieście', 4, False)
(23, 'Inne', 4, False)
(24, 'Urodziny', 5, True)


Convert data from csv file

In [27]:
xls['Opis operacji'] = xls['Opis operacji'].str.split(" ").str[0]

xls = xls.drop(columns=['Kategoria','Rachunek'])

# Convert the date format to YYYY-MM-DD
xls['Data operacji'] = pd.to_datetime(xls['Data operacji']).dt.strftime('%Y-%m-%d')

# Convert 'Kwota' column to numeric, coerce errors to NaN
xls['Kwota'] = pd.to_numeric(xls['Kwota'], errors='coerce')

# Drop rows with NaN values in 'Kwota' column
xls = xls.dropna(subset=['Kwota'])

<bound method DataFrame.to_string of     Data operacji        Opis operacji    Kwota
0      26/01/2025                 DODO   -66.98
1      26/01/2025              GABRIEL       20
2      25/01/2025                  JMP   -77.24
3      25/01/2025                DAWID      100
4      25/01/2025              GABRIEL       20
..            ...                  ...      ...
97     02/01/2025            CUKIERNIA       -2
98     02/01/2025              GABRIEL       30
99     01/01/2025              Spotify   -23.99
100    01/01/2025  Microsoft*Microsoft   -29.99
101    01/01/2025              NETCAMP  -173.87

[102 rows x 3 columns]>


Add data to database

In [ ]:
# Insert data into the database
cur.executemany(
    "INSERT INTO transactions (date, memo, amount) VALUES (%s, %s, %s)",
    xls[['Data operacji', 'Opis operacji', 'Kwota']].values.tolist()
)
conn.commit()
